In [28]:
df = spark.read.format("csv").option("header","true").load("Files/DimGeography.csv")
dfNSW = df.select(df.City, df.StateProvinceCode) \
             .where(df.StateProvinceCode == "NSW")
display(dfNSW)

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 03f6003c-23fd-4099-8728-49e24b56c4f3)

In [29]:
dfDE = df.select(df.City, df.StateProvinceCode.alias("State"), df.CountryRegionCode) \
         .where(df.CountryRegionCode == "DE")
display(dfDE)

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9c32e488-7550-4111-b8bb-56d0f69e5b14)

In [31]:
dfUnion = dfNSW.unionByName(dfDE, allowMissingColumns=True)
display(dfUnion)

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 16cadc65-6465-4519-998e-8074abea9f4c)

In [32]:
dfCustomer = spark.read.table("DimCustomer")
dfCustomer = dfCustomer.select("CustomerKey", "GeographyKey", "FirstName", "LastName")
display(dfCustomer)

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7fe7612e-8264-47e5-bd4d-7a26c532f4d0)

In [35]:
dfGeography = spark.read.table("DimGeography")
dfGeography = dfGeography.select(
    dfGeography.GeographyKey.alias("GeoKey"),
    dfGeography.City,
    dfGeography.EnglishCountryRegionName
)
display(dfGeography)

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 36a681d5-fc7e-41ea-9aa9-31373b7c0a5e)

In [41]:
dfJoin = dfCustomer.join(dfGeography, 
                         dfGeography.GeoKey == dfCustomer.GeographyKey, 
                         'full')

display(dfJoin)
display(dfJoin.where(dfJoin.GeographyKey.isNull()))
display(dfJoin.where(dfJoin.GeoKey.isNull()))

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 69b8dcee-027f-4b4c-851b-1d26dbf8322b)

SynapseWidget(Synapse.DataFrame, b34f5381-83b4-43bd-8cd0-453b77173519)

SynapseWidget(Synapse.DataFrame, eb8a1cab-e296-4d43-874b-aa594efdd81c)

In [50]:
display(dfJoin.groupBy("LastName").count().where("count > 1"))

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 52, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 16bfa469-2872-48e4-a546-fef4e12da466)

In [53]:
%%sql
SELECT City, StateProvinceCode, NULL as CountryRegionCode
FROM (
    SELECT City, StateProvinceCode
    FROM DimGeography
    WHERE StateProvinceCode = "NSW"
)
UNION ALL 
(
    SELECT City, StateProvinceCode, CountryRegionCode
    FROM DimGeography
    WHERE CountryRegionCode = "DE"
);

StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 55, Finished, Available, Finished)

<Spark SQL result set with 84 rows and 3 fields>

In [57]:
%%sql
SELECT CustomerKey, c.GeographyKey, FirstName, LastName,
       g.GeographyKey as GeoKey, City, EnglishCountryRegionName

FROM (
    SELECT CustomerKey, GeographyKey, FirstName, LastName
    FROM DimCustomer
) as c
FULL JOIN 
(
    SELECT GeographyKey, City, EnglishCountryRegionName
    FROM DimGeography
) as g
ON c.GeographyKey = g.GeographyKey



StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 60, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 7 fields>

In [79]:
%%sql
SELECT count(*)
FROM (
    SELECT 
       CustomerKey, c.GeographyKey, FirstName, LastName,
       g.GeographyKey as GeoKey, City, EnglishCountryRegionName
    FROM (
        SELECT CustomerKey, GeographyKey, FirstName, LastName
        FROM DimCustomer
    ) as c
    FULL JOIN 
    (
        SELECT GeographyKey, City, EnglishCountryRegionName
        FROM DimGeography
    ) as g
    ON c.GeographyKey = g.GeographyKey
)
-- WHERE GeographyKey IS NULL
-- WHERE GeoKey IS NULL
GROUP BY LastName
HAVING count(*) > 1


StatementMeta(, 4dfdd4c0-fe34-4083-813e-e3093df766d5, 82, Finished, Available, Finished)

<Spark SQL result set with 250 rows and 1 fields>